#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행
    * 성능 비교
    * 각 모델의 성능을 저장하는 별도 데이터 프레임을 만들고 비교
* 옵션 : 다음 사항은 선택사항입니다. 시간이 허용하는 범위 내에서 수행하세요.
    * 상위 N개 변수를 선정하여 모델링 및 성능 비교
        * 모델링에 항상 모든 변수가 필요한 것은 아닙니다.
        * 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교하세요.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것입니다.

## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.




* 함수 생성

In [ ]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
* 세부 요구사항
    - 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - 데이터프레임에 대한 기본 정보를 확인합니다.( .head(), .shape 등)

#### 1) 데이터 로딩

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_train.csv')

In [ ]:
# data에서 변수 subject는 삭제
data.drop(columns = 'subject', inplace=True)

#### 2) 기본 정보 조회

In [ ]:
data.shape

(5881, 562)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 25.2+ MB


In [ ]:
data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,...,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000
mean,0.274811,-0.017799,-0.109396,-0.603138,-0.509815,-0.604058,-0.628151,-0.525944,-0.605374,-0.465490,...,0.126955,-0.305883,-0.623548,0.008524,-0.001185,0.009340,-0.007099,-0.491501,0.059299,-0.054594
std,0.067614,0.039422,0.058373,0.448807,0.501815,0.417319,0.424345,0.485115,0.413043,0.544995,...,0.249176,0.322808,0.310371,0.339730,0.447197,0.608190,0.476738,0.509069,0.297340,0.278479
min,-0.503823,-0.684893,-1.000000,-1.000000,-0.999844,-0.999667,-1.000000,-0.999419,-1.000000,-1.000000,...,-0.965725,-0.979261,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980143
25%,0.262919,-0.024877,-0.121051,-0.992774,-0.977680,-0.980127,-0.993602,-0.977865,-0.980112,-0.936067,...,-0.021610,-0.541969,-0.845985,-0.122361,-0.294369,-0.481718,-0.373345,-0.811397,-0.018203,-0.141555
50%,0.277154,-0.017221,-0.108781,-0.943933,-0.844575,-0.856352,-0.948501,-0.849266,-0.849896,-0.878729,...,0.133887,-0.342923,-0.712677,0.010278,0.005146,0.011448,-0.000847,-0.709441,0.182893,0.003951
75%,0.288526,-0.010920,-0.098163,-0.242130,-0.034499,-0.262690,-0.291138,-0.068857,-0.268539,-0.013690,...,0.288944,-0.127371,-0.501158,0.154985,0.285030,0.499857,0.356236,-0.511330,0.248435,0.111932
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,0.977344,0.478157,1.000000


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다.


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [ ]:
y = data['Activity']
x = data.drop(columns = ['Activity'])

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

### (3) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y, stratify = y, random_state=42)

## **3. 기본 모델링**



* 세부 요구사항
    - 최소 4개 이상의 알고리즘을 적용하여 모델링을 수행한다.
    - 각 알고리즘별로 전체 변수로 모델링, 상위 N개 변수를 선택하여 모델링을 수행하고 성능 비교를 한다.
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

### (1) 알고리즘1 :

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

clf_SGD = SGDClassifier(random_state=42)
clf_SGD.fit(X_train, y_train)

predicted = clf_SGD.predict(X_val)
macro_f1_score = f1_score(y_val, predicted, average='macro')
print('Accuracy Score :', np.mean(predicted == y_val))
print("Macro F1 Score:", macro_f1_score)

Accuracy Score : 0.9850441876274643
Macro F1 Score: 0.985129560515665


### (2) 알고리즘2 :

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train, y_train)

predicted_logistic = clf_logistic.predict(X_val)
macro_f1_score_logistic = f1_score(y_val, predicted_logistic, average='macro')
print("Accuracy Score:", np.mean(predicted_logistic == y_val))
print("Macro F1 Score:", macro_f1_score_logistic)

Accuracy Score: 0.9836845683208701
Macro F1 Score: 0.9844251622523621


### (3) 알고리즘3 :

In [ ]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train, y_train)

predicted_svm = clf_svm.predict(X_val)
macro_f1_score_svm = f1_score(y_val, predicted_svm, average='macro')
print("SVM Accuracy Score:", np.mean(predicted_svm == y_val))
print("SVM Macro F1 Score:", macro_f1_score_svm)


SVM Accuracy Score: 0.9898028552005439
SVM Macro F1 Score: 0.9902892266717301


### (4) 알고리즘4 :

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_random_forest = RandomForestClassifier(random_state=42)
clf_random_forest.fit(X_train, y_train)

predicted_random_forest = clf_random_forest.predict(X_val)
macro_f1_score_random_forest = f1_score(y_val, predicted_random_forest, average='macro')
print("RandomForestClassifier Accuracy Score:", np.mean(predicted_random_forest == y_val))
print("RandomForestClassifier Macro F1 Score:", macro_f1_score_random_forest)

RandomForestClassifier Accuracy Score: 0.9789259007477906
RandomForestClassifier Macro F1 Score: 0.9792212105310995


## **4. 변수 중요도 선정 후 모델링**



* 세부 요구사항
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

In [ ]:
import joblib
feature_importances = joblib.load('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/feature1.pkl')

In [ ]:
tmp = feature_importances.groupby('feature_name').sum()

<ipython-input-132-fb9152dbe378>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = feature_importances.groupby('feature_name').sum()


In [ ]:
tmp.columns

Index(['importance_mission1', 'importance_mission2',
       'importportance_is_standing', 'importportance_is_sitting',
       'importportance_is_laying', 'importportance_is_walking',
       'importportance_is_walking_up', 'importportance_is_walking_down'],
      dtype='object')

In [ ]:
tmp['total_'] = tmp[tmp.columns].sum(axis=1)

In [ ]:
# 상위 20개만 뽑았음
topN = tmp.sort_values('total_', ascending=False).head(100).index

In [ ]:
# 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_train.csv')

y = data['Activity']
x = data.drop(columns = ['Activity'])

x_topN = x[topN]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_top20 = scaler.fit_transform(x_top20)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x_top20, y, stratify = y, random_state=42)

### (1) 알고리즘1 :

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

clf_SGD = SGDClassifier(random_state=42)
clf_SGD.fit(X_train, y_train)

predicted = clf_SGD.predict(X_val)
macro_f1_score = f1_score(y_val, predicted, average='macro')
print('Accuracy Score :', np.mean(predicted == y_val))
print("Macro F1 Score:", macro_f1_score)

Accuracy Score : 0.938137321549966
Macro F1 Score: 0.9391657988615528


### (2) 알고리즘2 :

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train, y_train)

predicted_logistic = clf_logistic.predict(X_val)
macro_f1_score_logistic = f1_score(y_val, predicted_logistic, average='macro')
print("Accuracy Score:", np.mean(predicted_logistic == y_val))
print("Macro F1 Score:", macro_f1_score_logistic)

Accuracy Score: 0.9449354180829368
Macro F1 Score: 0.9467040021036963


### (3) 알고리즘3 :

In [ ]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train, y_train)

predicted_svm = clf_svm.predict(X_val)
macro_f1_score_svm = f1_score(y_val, predicted_svm, average='macro')
print("SVM Accuracy Score:", np.mean(predicted_svm == y_val))
print("SVM Macro F1 Score:", macro_f1_score_svm)


SVM Accuracy Score: 0.9490142760027193
SVM Macro F1 Score: 0.9510191190085058


### (4) 알고리즘4 :

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_random_forest = RandomForestClassifier(random_state=42)
clf_random_forest.fit(X_train, y_train)

predicted_random_forest = clf_random_forest.predict(X_val)
macro_f1_score_random_forest = f1_score(y_val, predicted_random_forest, average='macro')
print("RandomForestClassifier Accuracy Score:", np.mean(predicted_random_forest == y_val))
print("RandomForestClassifier Macro F1 Score:", macro_f1_score_random_forest)

RandomForestClassifier Accuracy Score: 0.9762066621346023
RandomForestClassifier Macro F1 Score: 0.9757489629953868
